In [1]:
from student.tools import *
from student.agent import *

In [2]:
ag = MemoryAgent()
ag.run("Do not do a test run of the add memory tool")
ag.run("use the add memory tool once for a test run with the key 'test'")
ag.run("use the recall memory tool for a test run with the key 'test'")
ag.render_chat_html()

In [4]:
ag.tools['recall'].run(["['test']"])

'<tool>Recalled from memory with the stimuli \n\t<stimuli>["[\'test\']"]</stimuli>: \n\t\n        <memory id="14399432-e0b4-4b0f-8a41-e2d425577f35">\n            <stimuli>test</stimuli>\n            <content>This is a test memory entry.</content>\n        </memory>\n        \n        <memory id="9c19965f-89e2-4d6e-9956-fd69218e44ce">\n            <stimuli>test</stimuli>\n            <content>This is a test memory entry.</content>\n        </memory>\n        \n        <memory id="55012087-5bda-4b6a-9ade-564096e304c3">\n            <stimuli>test</stimuli>\n            <content>This is a test memory entry.</content>\n        </memory>\n        \n</tool>'

In [ ]:
ag = MemoryAgent()
ag.run("The secret password is zurich7891")
#ag.run("Three elefants tried to access the club but did not know the password to enter and left again.")
#ag.run("I want to access the club. What is the password?")
ag.render_chat_html()

In [5]:
ag.run("Why did you add the memory twice?")

"It appears that the memory was added multiple times due to an oversight in processing the request. Each time I executed the add memory tool, it created a new entry with the same content and stimuli. \n\nI can remove the duplicate entries if you'd like. Would you like me to proceed with that?"

In [6]:
ag.run("Yes")
ag.render_chat_html()

In [12]:
print(ag.memory.recall("test")[0])


        <memory id="14399432-e0b4-4b0f-8a41-e2d425577f35">
            <stimuli>test</stimuli>
            <content>This is a test memory entry.</content>
        </memory>
        
